In [ ]:
from torchvision.models import vit_b_32, ConvNeXt_Large_Weights
from torchvision.models import convnext_small, ConvNeXt_Small_Weights
import torch
from torch.utils.data import DataLoader
import sys
sys.path.append("..")
from utils import get_best_available_device
from dataset import ImageDataset
import matplotlib.pyplot as plt

# 1. Get dataset

In [ ]:
# BATCH_SIZE = 32
BATCH_SIZE = 1
device = get_best_available_device()
print("Using device:", device)

dataset = ImageDataset(path="../../data/training/", device="cpu", use_patches=False)
dataloader = iter(DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True))

In [ ]:
print(f"There are {len(dataloader)} images in the dataset.")

In [ ]:
batch = next(dataloader)
images, masks = batch

plt.imshow(images[0].numpy().transpose(1, 2, 0))
plt.imshow(masks[0].numpy().transpose(1, 2, 0), alpha=0.3)
plt.show()

# 2. Create model and load weights

In [ ]:
import json

with open('imagenet_labels.json', 'r') as f:
    labels = json.load(f)["labels"]


In [ ]:
# preprocess = weights.transforms()
# preprocess
# TODO do we remove preprocessing?

In [ ]:
with torch.no_grad():
    out = encoder(images)  
# len(encoder.x_int)
# list of channels in and channels out when upsampling
sizes = [(y.shape[1], x.shape[1])  for x,y in zip(encoder.x_int, encoder.x_int[1:])]
sizes.reverse()
# for int in encoder.x_int:
    # print(int.shape)
# out.shape
print("those are the sizes for the decoder inferred from the encoder", sizes)


In [ ]:
from swin_unet import SwinUNet

model_unet = SwinUNet()

In [ ]:
with torch.no_grad():
    out2 = model_unet(images)

In [ ]:
out2.shape